In [20]:
import pandas as pd
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import regex as re
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import GridSearchCV
#Hago las importaciones y me traigo los datos
total_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv")
total_data.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [21]:
#Lo primero es pasar los true y false (booleanos) a 0s y 1s
total_data["is_spam"] = total_data["is_spam"].apply(lambda x: 1 if x else 0).astype(int)
total_data.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1


In [4]:
#Quito los duplicados porque no añaden valor predictivo
print(total_data.shape)
total_data = total_data.drop_duplicates()
total_data = total_data.reset_index(inplace = False, drop = True)
total_data.shape

(2999, 2)


(2369, 2)

In [22]:
#Comprobamos cuántos spam y no spam tenemos antes de empezar
print(f"Spam: {len(total_data.loc[total_data.is_spam == 1])}")
print(f"No spam: {len(total_data.loc[total_data.is_spam == 0])}")

#Tenemos que aproximadamente un 10% de la muestra es spam. Aquí tengo que tener cuidado
#porque eso significa que un modelo que diga "no spam" constantemente estará acertando
#el 90% de las veces. Por ponerlo en lenguaje más preciso, tendré que vigilar los
#falsos negativos. En este caso, las urls de spam que no están siendo detectadas como
#spam

#Hay varias formas de solucionar esto. La más directa es penalizarle más cuando no
#detecte la clase minoritaria (spam en este caso) y
#model = SVC(kernel="linear", class_weight="balanced", random_state=42)
#model.fit(X_train, y_train)

Spam: 696
No spam: 2303


In [23]:
#Y ahora recurrimos a las expresiones regulares para estandartizar el formato (todo
#en minus, sin espacios en blanco, sin dobles espacios ni etiquetas)
def preprocess_text(text):
    text = re.sub(r'[^a-z ]', " ", text)
    text = re.sub(r'\s+[a-zA-Z]\s+', " ", text)
    text = re.sub(r'\^[a-zA-Z]\s+', " ", text)
    text = re.sub(r'\s+', " ", text.lower())
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", text)

    return text.split()

total_data["url"] = total_data["url"].apply(preprocess_text)
total_data.head()

,url,is_spam
0,"[https, briefingday, us, list, manage, com, un...",1
1,"[https, www, hvper, com]",1
2,"[https, briefingday, com, v, i]",1
3,"[https, briefingday, com, m, commentform]",0
4,"[https, briefingday, com, fan]",1


In [24]:
#Aquí ya estamos jugando con el significado base de las palabras. Ya no son solamente
#un conjunto de letras sino que se agrupan por significado con el lemmatizer.
#Por las mismas, palabras que no aporten significado como "the" etc se eliminan
#Palabas que tengan menos de 4 letras se eliminan también
#Con esto también reducimos la dimensionalidad porque en última instancia estamos
#reduciendo la cantidad total de palabras distitnas a tener en cuenta
download("wordnet")
lemmatizer = WordNetLemmatizer()

download("stopwords")
stop_words = stopwords.words("english")

def lemmatize_text(words, lemmatizer = lemmatizer):
    tokens = [lemmatizer.lemmatize(word) for word in words]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [word for word in tokens if len(word) > 3]
    return tokens

total_data["url"] = total_data["url"].apply(lemmatize_text)
total_data.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,url,is_spam
0,"[http, briefingday, list, manage, unsubscribe]",1
1,"[http, hvper]",1
2,"[http, briefingday]",1
3,"[http, briefingday, commentform]",0
4,"[http, briefingday]",1


In [25]:
#Y ahora parece que toca deshacer parte de lo caminado porque estamos devolviendo
#las palabras de tokens a una sola cadena de texto. El motivo es que TfidfVectorizer
#espera recibir texto plano, con lo cual una lista de palabras no le sirve
tokens_list = total_data["url"]
tokens_list = [" ".join(tokens) for tokens in tokens_list]

#Con respecto a los parámetros elegidos, tenemos que estamos tomando las 5000 palabras
#más frecuentes (hay que poner algún límite. Además, es muy probable que el vocabulario
#que resulte relevante en la toma de decisiones de clasificación sea mucho menor).
#
vectorizer = TfidfVectorizer(max_features = 5000, max_df = 0.8, min_df = 5)
X = vectorizer.fit_transform(tokens_list).toarray()
y = total_data["is_spam"]

X[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
#Entrenamiento y prueba. Como en todos los proyectos 80-20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [27]:

#Aquí es donde digo que, si bien es válido con SVC(kernel="linear", random_state=42)
#he querido probar con class_weight="balanced" para lo que comentábamos de hacer algo
#de contrapeso al hecho de que tenemos solamente un 10% de los datos que son spam
model = SVC(kernel="linear", class_weight="balanced", random_state=42)
model.fit(X_train, y_train)

SVC(class_weight='balanced', kernel='linear', random_state=42)

In [28]:
y_pred = model.predict(X_test)
y_pred

array([1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,

In [29]:
accuracy_score(y_test, y_pred)
print(f"Precisión: {accuracy_score(y_test, y_pred)}")
#Vaya. Pues parece que no hemos conseguido mejorar mucho frente a ese 89,7% que esperábamos

Precisión: 0.925


In [16]:
#Con una buena búsqueda de hiperparámetros seguro que se soluciona
#Seguramente haré dos, una muy general para encontrar el orden de magnitud que mejores
#resultados devuelva, y otra para afinar más dentro del rango bueno

#Empezamos con estos
hyperparams = {
    "C": [0.64, 0.8, 1, 1.2, 1.44],
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "degree": [1, 2, 3],
    "gamma": ["scale", "auto"]
}

#hyperparams = {
    #"C": [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    #"kernel": ["linear", "poly", "rbf", "sigmoid", "precomputed’"],
    #"degree": [1, 2, 3, 4, 5],
    #"gamma": ["scale", "auto"]
#}
#Tras 15 mins ha encontrado que los mejores son: {'C': 0.1, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}
#Los segundos mejores hiperparámetros que nos saca el gridsearch son: {'C': 0.64, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
grid = GridSearchCV(model, hyperparams, scoring = "accuracy", cv = 5)
grid

GridSearchCV(cv=5,
             estimator=SVC(class_weight='balanced', kernel='linear',
                           random_state=42),
             param_grid={'C': [0.64, 0.8, 1, 1.2, 1.44], 'degree': [1, 2, 3],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='accuracy')

In [17]:
grid.fit(X_train, y_train)

print(f"Best hyperparameters: {grid.best_params_}")

Best hyperparameters: {'C': 0.64, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}


In [19]:
#Genial! La optimización con gridsearch sí que nos ha llevado más lejos. Puede no
#parecer significativo, pero una performance del 96% de precisión es más del doble
#de buena que una del 89,7%, pues la primera fracasa en un 4% de los casos frente
#al 10,3% de la segunda
opt_model = SVC(C = 0.64, degree = 3, gamma = "scale", kernel = "poly", random_state = 42)
opt_model.fit(X_train, y_train)
y_pred = opt_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.959915611814346

In [ ]:
from pickle import dump

dump(model, open("/models/svm_classifier_C-1000_deg-1_gam-auto_ker-poly_42.sav", "wb"))